In [17]:
import pandas as pd
import numpy as np
import os, sys
import glob
import re
import hashlib
import pathlib
from tqdm import tqdm
#from sklearn.model_selection import train_test_split
from rxwgan.core import stratified_train_val_test_splits
from rxwgan.datasets import DownloadDataset
import warnings
warnings.filterwarnings("ignore")

In [2]:
token="b16fe0fc92088c4840a98160f3848839e68b1148"

In [30]:
#df = DownloadDataset(token).download('Imageamento', 'images', basepath='.')
df = DownloadDataset(token).download('imageamento', 'images', basepath='.')

100%|████████████████████████████████████████████████████████████████████████████████| 255/255 [00:29<00:00,  8.60it/s]


In [34]:
def expand_folder( path , extension):
    l = glob.glob(path+'/*.'+extension)
    l.sort()
    return l

In [35]:
def get_md5(path):
    return hashlib.md5(pathlib.Path(path).read_bytes()).hexdigest()


In [36]:
# this is the location of the raw chenzen data
base_data_raw_path = '/home/joao.pinto/public/brics_data/SantaCasa/imageamento/raw/images'

In [37]:
def prepare_my_table( path, target ):
    
    d = {
      'target'   : [],
      'image_ID' : [],
      'raw_image_path'     : [],
      'raw_image_md5'      : [],
    }


    
    for idx, path in enumerate(expand_folder(path, 'jpg')):
            
            image_filename = path.split('/')[-1]
            image_path = path
            d['target'].append(target)
            d['raw_image_path'].append(image_path)
            d['raw_image_md5'].append(get_md5(image_path))
            d['image_ID'].append(image_filename.replace('.png',''))
   
            
    return pd.DataFrame(d)


df = prepare_my_table(base_data_raw_path, 0)
df['run'] = 'SantaCasa_imageamento'


In [39]:
df.head()


,target,image_ID,raw_image_path,raw_image_md5,run
0,0,image_20220315_184023_414198.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,08de3b74c0ea27cff51de66a956c1cd4,SantaCasa_imageamento
1,0,image_20220315_184432_B11816.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,d85d703b36f43432da00f2904cb159eb,SantaCasa_imageamento
2,0,image_20220315_184701_E8A15F.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,63280de609a19562058afb008c046b13,SantaCasa_imageamento
3,0,image_20220315_184740_2D93D3.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,e4a0f37025ed430ed65745cdf593357c,SantaCasa_imageamento
4,0,image_20220315_185230_09A8AE.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,91bcaaa15198d0fdd9c36721c6d530eb,SantaCasa_imageamento


In [40]:
#df.to_csv('/home/jodafons/public/brics_data/Shenzhen/raw/Shenzhen_table_from_raw.csv')
df.to_csv('user.joao.pinto_SantaCasa_imageamento_table_from_raw.csv')


# Split K-Folds

In [41]:

seed = 512
splits = stratified_train_val_test_splits(df,10,seed)
pickle.dump(splits, open('splits.pkl','wb') )


bins selected for val: [1, 2, 3, 4, 5, 6, 7, 8, 9]


100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 4891.00it/s]


bins selected for val: [0, 2, 3, 4, 5, 6, 7, 8, 9]


100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 6393.76it/s]


bins selected for val: [0, 1, 3, 4, 5, 6, 7, 8, 9]


...

bins selected for val: [0, 1, 2, 3, 4, 5, 6, 7, 9]


100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 7143.97it/s]


bins selected for val: [0, 1, 2, 3, 4, 5, 6, 7, 8]


100%|██████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 6778.37it/s]


NameError: name 'pickle' is not defined

In [42]:
df_splitted = None

for test in range(10):
    for sort in range(9):
        
        train_index = splits[test][sort][0]
        val_index = splits[test][sort][1]
        test_index = splits[test][sort][2]
        
        df_train = df.iloc[train_index]
        df_train['test'] = test
        df_train['sort'] = sort
        df_train['dataset'] = 'train'
        
        df_val = df.iloc[val_index]
        df_val['test'] = test
        df_val['sort'] = sort
        df_val['dataset'] = 'val'       
 
        df_test = df.iloc[test_index]
        df_test['test'] = test
        df_test['sort'] = sort
        df_test['dataset'] = 'test'       

        if df_splitted is not None:
            df_splitted = pd.concat((df_splitted, df_train, df_val, df_test) )
        else:
            df_splitted = pd.concat((df_train, df_val, df_test) )
            
        df_splitted['type'] = 'real'

In [43]:
df_splitted.to_csv('user.joao.pinto_SantaCasa_imageamento_table_from_raw_splitted.csv')


In [44]:
df_splitted

,target,image_ID,raw_image_path,raw_image_md5,run,test,sort,dataset,type
0,0,image_20220315_184023_414198.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,08de3b74c0ea27cff51de66a956c1cd4,SantaCasa_imageamento,0,0,train,real
1,0,image_20220315_184432_B11816.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,d85d703b36f43432da00f2904cb159eb,SantaCasa_imageamento,0,0,train,real
3,0,image_20220315_184740_2D93D3.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,e4a0f37025ed430ed65745cdf593357c,SantaCasa_imageamento,0,0,train,real
4,0,image_20220315_185230_09A8AE.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,91bcaaa15198d0fdd9c36721c6d530eb,SantaCasa_imageamento,0,0,train,real
5,0,image_20220315_185346_EE823A.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,90aa14ef4eb038986011b36f933e85bd,SantaCasa_imageamento,0,0,train,real
...,...,...,...,...,...,...,...,...,...
38,0,image_20220317_155119_AB6FFB.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,19369365ab2bc94f07c80121961021d2,SantaCasa_imageamento,9,8,test,real
39,0,image_20220317_155333_253D0C.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,54d0bc620f4969dc19fca3f23ea7db4f,SantaCasa_imageamento,9,8,test,real
56,0,image_20220317_164325_305BC9.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,fb46e20503046ae10e09b374e0a2345a,SantaCasa_imageamento,9,8,test,real
75,0,image_20220321_144842_814C75.jpg,/home/joao.pinto/public/brics_data/SantaCasa/i...,f259228be579e99b222ea364d3c938df,SantaCasa_imageamento,9,8,test,real
